In [1]:
import pandas as pd
import numpy as np
import math
import random
#import statistics

In [2]:
#random.normalvariate(mu, sigma)
#mu is the mean, and sigma is the standard deviation

#A normal distribution with a mean of 0 and a standard deviation of 1

#np.random.normal(loc=0.0, scale=1.0, size=None)
#loc: Mean (“centre”) of the distribution
#scale:Standard deviation (spread or “width”) of the distribution.

In [3]:
def random_list(year, path):
    returns = np.zeros((path,year))  
    for i in range(0, path):
        P_i = np.random.normal(0, 1, year)
        returns[i] = P_i # Set each row of returns equal to the new R_i array
    return returns

As an alternative, please consider the following.
Let the random variables X ~ N(0,1) and Y ~ N(0,1) independently. 
Then the random variables X and $rhoX + \sqrt{(1 - rho^2)}$ Y are both distributed N(0,1), but are now correlated with correlation rho.

In [4]:
def np_random_list(year,path,rho):
    # rho(p) :correlation coefficient
    a1 = random_list(year, path)
    a2 = random_list(year, path)
    _rho = math.sqrt(1-math.pow(rho,2))
    a3  = rho*a1 + _rho *a2
    return(a1,a2,a3)

In [7]:
def correlated_ts(year,price0,path,a1,a2,a3,drift,volatility):
    
    logcash = [math.log(price0)]
    logequit = [math.log(price0)]
    logbond = [math.log(price0)]
    
    for i in range(year):
       
        logcash.append(logcash[-1]   + drift[0])
        logequit.append(logequit[-1] + drift[1] - volatility[1] * volatility[1]/2 + volatility[1]* a1[path][i])
        logbond.append(logbond[-1]   + drift[2] - volatility[2] * volatility[2]/2 + volatility[2]* a3[path][i])

    return(logcash[1:],logequit[1:],logbond[1:])

In [ ]:
# useless
price_log_cash= []
price_log_equit = []
price_log_bond = []
    
price2 = []
matrix= np.array([[1,0,0],
                  [0,1, rho],
                  [0,rho,1]])
ap =  math.exp(rho*volatility[1]*volatility[2]*year)
a  =  ap-1

In [8]:

drift = [0,5.625/100,0.00674642]
volatility = [0,0.15,0.0222]
rho = 0.2

year = 2
price0 = 1
path = 1000000

In [9]:
(a1,a2,a3)= np_random_list(year, path,rho)

In [10]:
all_matrix = np.zeros((path,3,year))

for i in range(path):
    (_cash,_equit,_bond) = correlated_ts(year,price0,i,a1,a2,a3,drift,volatility)
    all_matrix[i] =[_cash,_equit,_bond]
    
np.exp(all_matrix,all_matrix);

In [11]:
equit_index = 1
equit_matrix = all_matrix[:,equit_index]
equit_final = all_matrix[:,equit_index,-1]

bond_index  = 2
bond_matrix = all_matrix[:,bond_index ]
bond_final = all_matrix[:,bond_index ,-1]

In [12]:
mean_equit = np.mean(equit_final)
mean_bond = np.mean(bond_final)

mean_equit_bond = np.mean( equit_final * bond_final)

mu_eq_est = (math.log(mean_equit/price0))/year 

mu_bond_est = (math.log(mean_bond/price0))/year 

cov = math.log((mean_equit_bond/(mean_equit*mean_bond)))/year

rho_est = cov/(volatility[1]*volatility[2])

In [13]:
print "mean equit",mean_equit, "  ",price0*math.exp(drift[equit_index]*year)
print "mean bond",mean_bond,"   ",price0*math.exp(drift[bond_index]*year)
print "mu equity",mu_eq_est,"  ",drift[equit_index],"            error: ", 100*(mu_eq_est-drift[equit_index])/drift[equit_index],"%"
print "mu bond  ",mu_bond_est," ",drift[bond_index],"        error: ",100*(mu_bond_est-drift[bond_index])/drift[bond_index],"%"
print "cov      ",cov," ", rho*volatility[1]*volatility[2],"          error: ",100*(cov- rho*volatility[1]*volatility[2])/ rho*volatility[1]*volatility[2],"%"
print "rho      ",rho_est,"    ", rho,"              error: ",100*((rho_est - rho)/rho),"%"

mean equit 1.11940433846    1.11907225691
mean bond 1.01358747751     1.01358427916
mu equity 0.0563983515855    0.05625             error:  0.263736151962 %
mu bond   0.00674799773842   0.00674642         error:  0.0233863058638 %
cov       0.000666383044438   0.000666           error:  6.37768989395e-07 %
rho       0.20011502836      0.2               error:  0.0575141798912 %
